In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

plt.style.use('seaborn-colorblind')

In [2]:
enem_train = pd.read_csv("train.csv")
enem_train.drop('Unnamed: 0', axis=1, inplace=True)

In [3]:
enem_test = pd.read_csv("test.csv")
enem_test.columns

Index(['NU_INSCRICAO', 'CO_UF_RESIDENCIA', 'SG_UF_RESIDENCIA', 'NU_IDADE',
       'TP_SEXO', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO',
       'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 'IN_TREINEIRO',
       'TP_DEPENDENCIA_ADM_ESC', 'IN_BAIXA_VISAO', 'IN_CEGUEIRA', 'IN_SURDEZ',
       'IN_DISLEXIA', 'IN_DISCALCULIA', 'IN_SABATISTA', 'IN_GESTANTE',
       'IN_IDOSO', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
       'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT',
       'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'TP_LINGUA',
       'TP_STATUS_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3',
       'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO', 'Q001', 'Q002',
       'Q006', 'Q024', 'Q025', 'Q026', 'Q027', 'Q047'],
      dtype='object')

In [4]:
#Ok, nem tentar brincar com coluna que não está no treino
#E ja vou tirar os códigos das provas
enem_train = enem_train[['NU_INSCRICAO', 'NU_NOTA_MT','CO_UF_RESIDENCIA', 'SG_UF_RESIDENCIA', 'NU_IDADE',
       'TP_SEXO', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO',
       'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 'IN_TREINEIRO',
       'TP_DEPENDENCIA_ADM_ESC', 'IN_BAIXA_VISAO', 'IN_CEGUEIRA', 'IN_SURDEZ',
       'IN_DISLEXIA', 'IN_DISCALCULIA', 'IN_SABATISTA', 'IN_GESTANTE',
       'IN_IDOSO', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
       'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'TP_LINGUA',
       'TP_STATUS_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3',
       'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO', 'Q001', 'Q002',
       'Q006', 'Q024', 'Q025', 'Q026', 'Q027', 'Q047']]

## Conhecendo `NU_NOTA_MT`

In [5]:
enem_train.query('NU_NOTA_MT == 0').shape[1]

44

In [ ]:
teste = enem_train.query('NU_NOTA_MT != 0')
teste['NU_NOTA_MT'].describe()

In [ ]:
sns.distplot(enem_train['NU_NOTA_MT']);

In [ ]:
print("Skewness: %f" % enem_train['NU_NOTA_MT'].skew())
print("Kurtosis: %f" % enem_train['NU_NOTA_MT'].kurt())

## Trabalhar com os floats
* NU_NOTA_COMP1
* NU_NOTA_COMP2
* NU_NOTA_COMP3
* NU_NOTA_COMP4
* NU_NOTA_COMP5
* NU_NOTA_REDACAO
* NU_NOTA_CN
* NU_NOTA_CH
* NU_NOTA_LC
* NU_NOTA_MT
* NU_IDADE

In [ ]:
floats = enem_train[['NU_NOTA_COMP1','NU_NOTA_COMP2','NU_NOTA_COMP3','NU_NOTA_COMP4','NU_NOTA_COMP5',
                     'NU_NOTA_REDACAO','NU_NOTA_MT','NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_IDADE']]

pd.DataFrame({'tipos':floats.dtypes, 
              'missing':floats.isna().sum()/floats.shape[0]*100})

In [ ]:
#para entender esses missing:
aux = enem_train[['TP_PRESENCA_CN','NU_NOTA_MT','NU_NOTA_CN']]
aux.query('TP_PRESENCA_CN == 0').head()

In [ ]:
#para entender esses missing:
aux = enem_train[['TP_PRESENCA_CH','TP_PRESENCA_LC','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_REDACAO']]
aux.query('TP_PRESENCA_CH == 0').head()

In [ ]:
enem_train[['NU_NOTA_MT','NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_REDACAO',
            'NU_NOTA_COMP1','NU_NOTA_COMP2','NU_NOTA_COMP3','NU_NOTA_COMP4','NU_NOTA_COMP5']].describe()

In [ ]:
sns.boxplot(data=enem_train[['NU_NOTA_MT','NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_REDACAO']])
plt.xticks(rotation=30, ha='right');

In [ ]:
sns.boxplot(data=enem_train[['NU_NOTA_COMP1','NU_NOTA_COMP2','NU_NOTA_COMP3','NU_NOTA_COMP4','NU_NOTA_COMP5']])
plt.xticks(rotation=30, ha='right');

In [ ]:
plt.figure(figsize=(10,4))
sns.distplot(enem_train['NU_NOTA_CN'].dropna(), hist=False, color='green')
sns.distplot(enem_train['NU_NOTA_CH'].dropna(), hist=False, color='red')
sns.distplot(enem_train['NU_NOTA_LC'].dropna(), hist=False, color='blue')
sns.distplot(enem_train['NU_NOTA_MT'].dropna(), hist=False, color='purple')
sns.distplot(enem_train['NU_NOTA_REDACAO'].dropna(), hist=False, color='yellow')
plt.xlabel("")
plt.show()

In [ ]:
sns.distplot(enem_train['NU_IDADE'])

In [ ]:
plt.figure(figsize=(7,6))
sns.heatmap(floats.corr(method='pearson'), cmap='YlOrRd', annot=True, fmt='.2f');

In [ ]:
#correlação das features com 'NU_NOTA_MT'
correlacao=floats.corr()
print(correlacao['NU_NOTA_MT'])

In [ ]:
enem_train_copy = enem_train.copy()

# 1. Baseline

Random Forest usando apenas **Notas**

In [ ]:
bl = enem_train_copy[['NU_NOTA_MT','NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_REDACAO']]
bl.fillna(-1, inplace=True)

In [ ]:
bl_y = bl['NU_NOTA_MT']
bl_x = bl.drop('NU_NOTA_MT', axis=1)

bl_x_train, bl_x_val, bl_y_train, bl_y_val = train_test_split(bl_x, bl_y, test_size=0.25, random_state=42)

In [ ]:
random_forest = RandomForestRegressor(criterion='mse', max_depth=8, n_jobs=-1, random_state=0)
random_forest.fit(bl_x_train, bl_y_train)

bl_y_pred = random_forest.predict(bl_x_val)

mae=metrics.mean_absolute_error(bl_y_val, bl_y_pred)
rmse=np.sqrt(metrics.mean_squared_error(bl_y_val, bl_y_pred))

print(mae,rmse)

In [ ]:
X_test = enem_test[bl_x.columns]

X_test.fillna(-1, inplace=True)

In [ ]:
y_pred = random_forest.predict(X_test)

In [ ]:
bl_answer = pd.DataFrame({'NU_INSCRICAO':enem_test.NU_INSCRICAO,
                        'TP_PRESENCA_CN':enem_test.TP_PRESENCA_CN,
                        'NU_NOTA_MT':y_pred})
bl_answer.head()

In [ ]:
tamanho = bl_answer.shape[0]
tamanho

In [ ]:
i=3

for i in range(0,tamanho):
    if (bl_answer.iloc[i:1+i,1:2] == 0).bool(): #ver os ausentes
        bl_answer.iloc[i:1+i,2:3] = np.nan
        i+=3

bl_answer.head()

In [ ]:
bl_answer.NU_NOTA_MT.describe()

In [ ]:
bl_answer.drop('TP_PRESENCA_CN', axis=1, inplace=True)
bl_answer.to_csv('answer.csv', index=False)

Codenation: 93.6457%

## Trabalhar com categóricas

In [ ]:
enem_train.columns

In [ ]:
categoricas_dig = enem_train[['NU_NOTA_MT','TP_SEXO', 'Q001', 'Q002', 'Q006', 'Q024', 'Q025', 'Q026', 
                              'Q027', 'Q047']]

In [ ]:
categoricas_dig.isna().sum().sort_values(ascending=False).head(3)

In [ ]:
sns.boxplot(categoricas_dig.TP_SEXO, categoricas_dig.NU_NOTA_MT)

**OBS:** Todos os 'TP_SEXO', 'Q001', 'Q002', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047' influenciam a nota de matemática pelos boxplot!!!

**OBS2**: Q027 muitos nulos

In [ ]:
categoricas_num = enem_train[['NU_NOTA_MT', 'CO_UF_RESIDENCIA',
       'NU_IDADE', 'TP_COR_RACA', 'TP_NACIONALIDADE',
       'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO',
       'IN_TREINEIRO', 'TP_DEPENDENCIA_ADM_ESC', 'IN_BAIXA_VISAO',
       'IN_CEGUEIRA', 'IN_SURDEZ', 'IN_DISLEXIA', 'IN_DISCALCULIA',
       'IN_SABATISTA', 'IN_GESTANTE', 'IN_IDOSO','TP_LINGUA', 'TP_STATUS_REDACAO']]

In [ ]:
categoricas_num.TP_ANO_CONCLUIU.value_counts()

In [ ]:
sns.boxplot(categoricas_num.TP_COR_RACA, categoricas_num.NU_NOTA_MT);

In [ ]:
sns.boxplot(categoricas_num.TP_ESCOLA, categoricas_num.NU_NOTA_MT);

In [ ]:
(categoricas_num.isna().sum()/categoricas_num.shape[0]*100).sort_values(ascending=False).head(6)

**OBS**: 'TP_NACIONALIDADE', 'IN_BAIXA_VISAO', 'IN_CEGUEIRA', 'IN_SURDEZ', 'IN_DISLEXIA', 'IN_DISCALCULIA',
       'IN_SABATISTA', 'IN_GESTANTE', 'IN_IDOSO', 'TP_STATUS_REDACAO' tem poucos valores contrastantes (5, 15...) não serve pra modelo.

**OBS2**: TP_ST_CONCLUSAO, TP_ANO_CONCLUIU, TP_ENSINO, IN_TREINEIRO aparentemente não influenciam na nota de matemática

**OBS3**: 'TP_ENSINO','TP_DEPENDENCIA_ADM_ESC' muitos NAN e não saberia lidar com eles

**Formando um dataframe**

In [ ]:
df_train = enem_train[['NU_NOTA_MT', 'NU_IDADE', 'TP_COR_RACA','TP_ESCOLA','TP_LINGUA',
                       'NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_REDACAO',
                       'TP_SEXO','Q001', 'Q002', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047']]

df_train.head()

0	Não declarado
1	Branca
2	Preta
3	Parda
4	Amarela
5	Indígena 
6	Não dispõe da informação


1	Não Respondeu
2	Pública
3	Privada
4	Exterior


In [ ]:
df_train['TP_SEXO'] = df_train['TP_SEXO'].map({'F':0, 'M':1}) 

df_train['TP_COR_RACA'] = df_train['TP_COR_RACA'].map({0:'Outros', 6:'Outros', 1:'Branca', 2:'Outros', 
                                                       3:'Outros', 4:'Branca', 5:'Outros'})

df_train['TP_ESCOLA'] = df_train['TP_ESCOLA'].map({1:'Outras', 2:'Outras', 3:'Privada', 4:'Outras'}) 

In [ ]:
df_train = pd.get_dummies(df_train)

In [ ]:
(df_train.isna().sum()/df_train.shape[0]*100).sort_values(ascending=False).head(6)

In [ ]:
df_train.fillna(-1, inplace=True)

In [ ]:
df_train.head()

In [ ]:
print('tenho',df_train.shape[1],'features')

In [ ]:
y = df_train['NU_NOTA_MT']
X = df_train.drop('NU_NOTA_MT', axis=1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
random_forest = RandomForestRegressor(criterion='mse', max_depth=8, n_jobs=-1, random_state=0)
random_forest.fit(X_train, y_train)

y_pred = random_forest.predict(X_val)

mae=metrics.mean_absolute_error(y_val, y_pred)
rmse=np.sqrt(metrics.mean_squared_error(y_val, y_pred))

print(mae,rmse)

In [ ]:
X_test = enem_test[['NU_IDADE', 'TP_COR_RACA','TP_ESCOLA','TP_LINGUA',
                    'NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_REDACAO',
                    'TP_SEXO','Q001', 'Q002', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047']]

X_test['TP_SEXO'] = X_test['TP_SEXO'].map({'F':0, 'M':1}) 

X_test['TP_COR_RACA'] = X_test['TP_COR_RACA'].map({0:'Outros', 6:'Outros', 1:'Branca', 2:'Outros', 
                                                       3:'Outros', 4:'Branca', 5:'Outros'})

X_test['TP_ESCOLA'] = X_test['TP_ESCOLA'].map({1:'Outras', 2:'Outras', 3:'Privada', 4:'Outras'})

X_test = pd.get_dummies(X_test)

X_test.fillna(-1, inplace=True)

In [ ]:
y_pred = random_forest.predict(X_test)

In [ ]:
answer = pd.DataFrame({'NU_INSCRICAO':enem_test.NU_INSCRICAO,
                        'TP_PRESENCA_CN':enem_test.TP_PRESENCA_CN,
                        'NU_NOTA_MT':y_pred})
answer.head()

In [ ]:
tamanho = answer.shape[0]
tamanho

In [ ]:
i=3

for i in range(0,tamanho):
    if (answer.iloc[i:1+i,1:2] == 0).bool(): #ver os ausentes
        answer.iloc[i:1+i,2:3] = np.nan
        i+=3

answer.head()

In [ ]:
answer.NU_NOTA_MT.describe()

In [ ]:
answer.drop('TP_PRESENCA_CN', axis=1, inplace=True)
answer.to_csv('answer.csv', index=False)

Codenation: 93.7467%